<div style="height:100px">
    <div style="float: left"><img src='../images/osr-logo.png' width=150px /></div>
    <div style="float: left; margin-left:30px"><span style="font-weight: normal; font-size: 30pt; color:#666666">密码工程实验指引</span></div>
    <img src='../images/osr-line.jpg' style="width:100%; height: 5px" />
</div>


# 汉明重量泄露模型

## 实验目的

通过实验，理解汉明重量泄露模型，掌握使用汉明重量模型分析能量曲线中的数据依赖性。

## 实验要求

- 编写将数据从内存转移到寄存器的程序，下载烧写到 `OSR-Mega2560` 或 `OSR-407`开发板；
- 使用osrtoolkit示波器模块和波形存储类采集能量曲线；
- 使用汉明重量模型分析能量曲线中的数据依赖性。

## 实验环境需求

- **硬件需求**
    + OSR-Mega2560 或 OSR-407开发板
    + Pico 3206D示波器
- **软件需求**
    + Arduino IDE
    + python osrtoolkit库

## 实验内容

### 汉明重量模型

在能量分析攻击中, 通常必须要将操作数映射为能量消耗值, 汉明重量模型和汉明距离模型是在能量分析中最常用的模型。

汉明重量模型假设能量消耗与被处理的数据中比特'1'的个数呈线性关系，即 $L = a\cdot HW(x) + b +r $，其中，$x$为中间值，$HW(\cdot)$ 为汉明重量函数，即计算数值中1的个数，$a$为系数，$b$为常数，$c$ 为随机噪声。

而汉明距离模型假设能量消耗与寄存器刷新前后数据的比特翻转个数呈线性关系，即 $L = a\cdot HD(x, y) + b +r $，其中，$x$为寄存器刷新前存储的中间值，$y$为寄存器刷新后存储的中间值，$HD(\cdot)$ 为汉明距离函数，即$0\rightarrow 1$和$1\rightarrow 0$的比特个数，$a$为系数，$b$为常数，$c$ 为随机噪声。

## 实验过程

### 导入osrtoolkit示波器模块和波形存储类

In [1]:
from osrtoolkit.oscilloscope import Pico3000

### 配置示波器控制模块

#### 示波器的参数选取可以参考下图：

   <img src="../images/HW_pico3000.png" width="500px" align="left" >

In [2]:
pico = Pico3000()  # 声明pico示波器对象
pico.form  # 打开pico示波器配置界面

### 与 TOE 进行通信

In [4]:
import serial
import numpy as np
import time

In [9]:
mega = serial.Serial('com11', 115200, timeout=1)

In [10]:
pico.arm()

True

In [11]:
mega.write(b'\x01')

1

### 测量微控制器将8比特数据从内存移动到寄存器时的能量消耗

对8比特数据，使用全部256个不同的数值。对于256个数值中的每一个，记录了1000次微控制器的能量消耗，并求出均值。

In [25]:
import matplotlib.pyplot as plt
from tqdm import tnrange
import time

In [26]:
%matplotlib widget

In [27]:
plt.rcParams['figure.figsize'] = [20,4]  # 调整图片尺寸

In [28]:
a = np.empty((256,3000), dtype = np.float) 

In [29]:
for i in tnrange(256):
    pico.arm()
    for j in range(1000):
        mega.write(bytes([i]))
        mega.read_all()
    time.sleep(0.05)
    trs = pico.acquire_samples()
    a[i] = np.mean(trs[1],axis=0)

将微处理器处理的8比特数据按汉明重量进行分类，并求出每一类数据对应的能量消耗的均值。

In [34]:
hw = np.empty((256), dtype = np.float) 
b = np.empty((9,3000), dtype = np.float) 

In [31]:
for i in range(256):
    c=str(bin(i))
    hw[i]=c.count('1')
    
for i in range(9):
    index = np.where(hw==i)
    b[i]=np.mean(a[index],axis=0)

In [32]:
plt.figure()
plt.plot(trs[0][0])
plt.plot(b.T)
plt.legend(['trigger','0','1','2','3','4','5','6','7','8'], loc='upper left')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

放大$[250,350]$区间的图像，我们可以清楚地看到对不同汉明重量的数据进行操作时，微控制器能量的消耗具有不同的均值。

In [33]:
plt.figure()
plt.plot(trs[0][0,250:350])
plt.plot(b[:,250:350].T)
plt.legend(['trigger','0','1','2','3','4','5','6','7','8'], loc='upper left')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

结果表明，寄存器变化带来的能量消耗符合汉明重量模型。

<img src='../images/osr-line.jpg' style="width:100%; height: 5px" />
<span style="font-weight: normal; font-size: 20pt; color:#666666">联系我们</span>

<span style="font-weight: normal; font-size: 18pt; color:#666666">深圳市纽创信安科技开发有限公司 (Open Security Research, Inc.)</span>
<div>
    <div style="float: left">
        <img src="../images/osr-code.png" width='150px'>
    </div>
    <div style="float: left; margin-left:30px">
        <h5>网址: <a href="https://www.opsefy.com/">www.opsefy.com</a> </h5>
        <h5>邮箱: <a href="mailto:sales@opsefy.com">sales@opsefy.com</a></h5>
        <h5>电话: +86 755 8695 0263</h5>
        <h5>
            地址：深圳市南山区粤海街道高新南 10 道深圳湾生态园 12 栋裙楼 8 楼 29-31 室<br/>
                 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
                北京市海淀区中关村东路 66 号世纪科贸大厦 B 座 2703 室
        </h5>
    </div>
</div>
